In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
# Random
import random
random.seed = 42
# Thread management (sleep purposes)
import time

import os
import selenium
# Selenium specific libraries
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

import pickle

In [2]:
if not os.path.exists("portales/indomio"):
  os.makedirs("portales/indomio")

In [14]:
configurations = {
    'url' : 'https://www.indomio.es/'}
service = Service(executable_path='chromedriver.exe')

options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36")
#options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

In [15]:
service = Service(executable_path='chromedriver.exe')
driver = webdriver.Chrome(service=service, options=options)

driver.get(configurations["url"])
driver.maximize_window()
time.sleep(np.random.normal(2,0.3)) # segundos

driver.save_screenshot("portales/indomio/00-homepage.png")

True

In [17]:
driver.find_element(by=By.ID, value="didomi-notice-agree-button").click()
time.sleep(np.random.normal(2,0.3)) # segundos
driver.save_screenshot("portales/fotocasa/01-homepage-no-cookies.png")

False

In [20]:
campo_busqueda = driver.find_element(by=By.CLASS_NAME, value="nd-autocomplete__input")
time.sleep(np.random.normal(2,0.3)) # segundos
consulta = "valencia poblacion"
campo_busqueda.send_keys(consulta)
time.sleep(np.random.normal(2))
# Simular la presión de la tecla Enter para realizar la búsqueda (si es necesario)
campo_busqueda.send_keys(Keys.RETURN)

time.sleep(np.random.normal(2)) # segundos
driver.save_screenshot("portales/indomio/02-homepage-no-cookies.png")

True

In [22]:
enlace = driver.find_element(by=By.CLASS_NAME, value= 'nd-accordion')
enlaces_hijos = enlace.find_elements(by=By.CLASS_NAME, value='im-zones__accordionLabelText')
distritos = []
for enlace_hijo in enlaces_hijos:
    distritos.append(enlace_hijo.get_attribute('title'))

In [29]:
print(distritos)

['Todo el municipio', 'Camins al Grau', 'El Pla del Real', 'Algirós', "Pobles de l'Oest", 'Quatre Carreres', 'Poblats Marítims', 'Benimaclet, Camí de Vera', 'Campanar, Sant Pau, El Calvari', 'Benicalap, Ciutat Fallera', 'Ciutat Vella', 'Pobles del Sud', "L'Olivereta", 'Rascanya', 'Sant Isidre, Patraix, Favara, Vara de Quart', "L'Eixample", 'Pobles del Nord', 'Extramurs', 'Jesús', 'La Saïdia']


In [30]:
enlaces_hijos_2 = enlace.find_elements(by=By.CLASS_NAME, value='nd-accordion__subMenu')
barrios = []
for i in range(len(enlaces_hijos_2)):
    enlaces_hijos_3 = enlaces_hijos_2[i].find_elements(by=By.CLASS_NAME, value='nd-checkbox__text')
    barrios.append(enlaces_hijos_3)

In [31]:
print(barrios)

[[<selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e9029690735ad4cd2", element="f.F0018DAC1770468CCEFD87CB179DC793.d.F7EF673C0501295AE7B20040EC15603E.e.267")>, <selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e9029690735ad4cd2", element="f.F0018DAC1770468CCEFD87CB179DC793.d.F7EF673C0501295AE7B20040EC15603E.e.268")>, <selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e9029690735ad4cd2", element="f.F0018DAC1770468CCEFD87CB179DC793.d.F7EF673C0501295AE7B20040EC15603E.e.269")>, <selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e9029690735ad4cd2", element="f.F0018DAC1770468CCEFD87CB179DC793.d.F7EF673C0501295AE7B20040EC15603E.e.270")>, <selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e9029690735ad4cd2", element="f.F0018DAC1770468CCEFD87CB179DC793.d.F7EF673C0501295AE7B20040EC15603E.e.271")>], [<selenium.webdriver.remote.webelement.WebElement (session="36d98568b789d24e902969

In [32]:
barriadas = []
for i in range(len(barrios)):
    for j in range(len(barrios[i])):
        barriadas.append(barrios[i][j].text)

In [33]:
barriadas

['Camí Fondo',
 'Albors',
 'La Creu del Grau',
 'Penya-roja',
 'Aiora',
 'Ciutat Universitària',
 'Mestalla',
 'Exposició',
 'Jaume Roig',
 'Ciutat Jardí',
 'La Carrasca',
 "L'Amistat",
 "L'Illa Perduda",
 'La Bega Baixa',
 'Benimàmet',
 'Beniferri',
 'Ciutat de les Arts i de les Ciències',
 'La Punta',
 'En Corts',
 'Montolivet',
 'Na Rovella',
 'Fonteta de Sant Lluís',
 'Malilla',
 'El Cabanyal-El Canyamelar',
 'El Grau',
 'Natzaret',
 'La Malva-rosa',
 'Beteró',
 'Benimaclet',
 'Camí de Vera',
 'Sant Pau',
 'Campanar',
 'El Calvari',
 'Les Tendetes',
 'Ciutat Fallera',
 'Benicalap',
 'El Carme',
 'Sant Francesc',
 'El Pilar',
 'La Xerea',
 'El Mercat',
 'La Seu',
 'Pinedo',
 'El Palmar',
 'La Torre',
 "El Castellar-L'Oliveral",
 'El Saler',
 "El Forn d'Alcedo",
 'El Perellonet',
 'Faitanar',
 'Nou Moles',
 'Soternes',
 'La Llum',
 'La Fontsanta-Tres Forques',
 'Orriols',
 'Sant Llorenç',
 'Torrefiel',
 'Sant Isidre',
 'Patraix',
 'Favara',
 'Vara de Quart',
 'Safranar',
 'Gran Via',

In [42]:
from unidecode import unidecode
for i in range(len(barriadas)):
    barriadas[i] = barriadas[i].replace(" ", "-").lower().replace("'","-").replace('*','-')
    barriadas[i] = unidecode(barriadas[i])

In [35]:
with open('barrios_vlc.pkl', 'wb') as f:
    pickle.dump(barriadas, f)

In [36]:
# Cargar el diccionario desde el archivo
with open('barrios_vlc.pkl', 'rb') as f:
    barriadas = pickle.load(f)

In [43]:
barriadas

['cami-fondo',
 'albors',
 'la-creu-del-grau',
 'penya-roja',
 'aiora',
 'ciutat-universitaria',
 'mestalla',
 'exposicio',
 'jaume-roig',
 'ciutat-jardi',
 'la-carrasca',
 'l-amistat',
 'l-illa-perduda',
 'la-bega-baixa',
 'benimamet',
 'beniferri',
 'ciutat-de-les-arts-i-de-les-ciencies',
 'la-punta',
 'en-corts',
 'montolivet',
 'na-rovella',
 'fonteta-de-sant-lluis',
 'malilla',
 'el-cabanyal-el-canyamelar',
 'el-grau',
 'natzaret',
 'la-malva-rosa',
 'betero',
 'benimaclet',
 'cami-de-vera',
 'sant-pau',
 'campanar',
 'el-calvari',
 'les-tendetes',
 'ciutat-fallera',
 'benicalap',
 'el-carme',
 'sant-francesc',
 'el-pilar',
 'la-xerea',
 'el-mercat',
 'la-seu',
 'pinedo',
 'el-palmar',
 'la-torre',
 'el-castellar-l-oliveral',
 'el-saler',
 'el-forn-d-alcedo',
 'el-perellonet',
 'faitanar',
 'nou-moles',
 'soternes',
 'la-llum',
 'la-fontsanta-tres-forques',
 'orriols',
 'sant-llorenc',
 'torrefiel',
 'sant-isidre',
 'patraix',
 'favara',
 'vara-de-quart',
 'safranar',
 'gran-via',

In [37]:
distritos = distritos[1:]

In [44]:
# se toma el tamaño de barrios,ese es el numero de barriadas que tiene cada distrito
# se crea un diccionario que tiene como clave el distrito y como valor una lista de barriadas
distrito_barrios = {}
j = 0
for i in range(len(distritos)):
    distrito_barrios[distritos[i]] = barriadas[j:j+len(barrios[i])]

    j = j + len(barrios[i])

In [50]:
def get_url(i,j):
    return f'https://www.indomio.es/venta-casas/valencia-capital/{barriadas[j]}/?pag={i}'
i = 0
j = 0
configurations = {
    'url': get_url(i,j),
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

response = requests.get(configurations['url'], headers={'User-Agent': configurations['user-agent']})

# Parsing the page
soup = BeautifulSoup(response.text, 'html.parser')
anuncios = soup.find_all('div', class_=['ad-preview'])
links = []
titulos_casas = []
anuncios = 0

for j in range(len(barriadas)):
    i = 0
    ratio = 0
    print(f'Barriada: {barriadas[j]}')
    configurations['url'] = get_url(i, j)
    
    while i <= ratio:
        print("Página actual:", configurations['url'])  
        response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})
        
        if response.status_code == 200:
            # Parsear el contenido HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            anuncios_header = soup.find('div', class_='in-realEstateListHeader__title').text
            matches = re.findall(r'\d+', anuncios_header)
            # Check if any matches were found
            if matches:
                # Extract the first match and convert it to an integer
                anuncios_number = int(matches[0])
                print("Number extracted:", anuncios_number)
            else:
                anuncios_number = 0
            ratio = anuncios_number / 25
            print(f'Ratio: {ratio}')
                                
            anuncios = soup.find_all('div', class_='nd-mediaObject__content')
            for anuncio in anuncios:
                enlace = anuncio.find('a').get('href')
                time.sleep(random.randint(1, 5) / 10)
                links.append(enlace)
                titulo = anuncio.find('a').get('title')
                time.sleep(random.randint(1, 5) / 10)
                titulos_casas.append(titulo)
                
            i += 1
            configurations['url'] = get_url(i, j)

        else:
            # Si la solicitud no fue exitosa, imprimir un mensaje de error y salir del bucle
            print("Error al obtener la página:", response.status_code)
            break

Barriada: cami-fondo
Página actual: https://www.indomio.es/venta-casas/valencia-capital/cami-fondo/?pag=0
Number extracted: 2
Ratio: 0.08
Barriada: albors
Página actual: https://www.indomio.es/venta-casas/valencia-capital/albors/?pag=0
Number extracted: 10
Ratio: 0.4
Barriada: la-creu-del-grau
Página actual: https://www.indomio.es/venta-casas/valencia-capital/la-creu-del-grau/?pag=0
Number extracted: 10
Ratio: 0.4
Barriada: penya-roja
Página actual: https://www.indomio.es/venta-casas/valencia-capital/penya-roja/?pag=0
Number extracted: 17
Ratio: 0.68
Barriada: aiora
Página actual: https://www.indomio.es/venta-casas/valencia-capital/aiora/?pag=0
Number extracted: 24
Ratio: 0.96
Barriada: ciutat-universitaria
Página actual: https://www.indomio.es/venta-casas/valencia-capital/ciutat-universitaria/?pag=0
Number extracted: 3
Ratio: 0.12
Barriada: mestalla
Página actual: https://www.indomio.es/venta-casas/valencia-capital/mestalla/?pag=0
Number extracted: 12
Ratio: 0.48
Barriada: exposicio
P

ConnectionError: HTTPSConnectionPool(host='www.indomio.es', port=443): Max retries exceeded with url: /venta-casas/valencia-capital/arrancapins/?pag=0 (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000001F5E472EE00>: Failed to resolve 'www.indomio.es' ([Errno 11001] getaddrinfo failed)"))

In [65]:
links

['https://www.indomio.es/anuncios/91743779/',
 'https://www.indomio.es/anuncios/94961259/',
 'https://www.indomio.es/agencias-inmobiliarias/382260/hogarabitat-servicios-inmobiliarios/',
 'https://www.indomio.es/agencias-inmobiliarias/380055/engel-volkers-valencia/',
 'https://www.indomio.es/agencias-inmobiliarias/479961/trencadis-real-estate/',
 'https://www.indomio.es/agencias-inmobiliarias/487523/live4life/',
 'https://www.indomio.es/agencias-inmobiliarias/377849/desiree-hato-gestion-de-inmuebles/',
 'https://www.indomio.es/agencias-inmobiliarias/488701/spaces-and-places-exclusive-property/',
 'https://www.indomio.es/agencias-inmobiliarias/486791/sky-consulting/',
 'https://www.indomio.es/agencias-inmobiliarias/380514/llave-maestra/',
 'https://www.indomio.es/agencias-inmobiliarias/382343/remax-urbe-ii/',
 'https://www.indomio.es/agencias-inmobiliarias/381809/gr2000/',
 'https://www.indomio.es/anuncios/95510051/',
 'https://www.indomio.es/anuncios/95280509/',
 'https://www.indomio.es

In [66]:
# eliminar los enlaces que sean 'https://www.indomio.es/agencias-inmobiliarias/....' ya que no son casas
links = [link for link in links if 'agencias-inmobiliarias' not in link]


In [67]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
unique_links = f7(links)
unique_titles = f7(titulos_casas)

In [19]:
import pickle

with open('links.pkl', 'wb') as f:
    pickle.dump(unique_links, f)

In [52]:
# Cargar el diccionario desde el archivo
with open('links.pkl', 'rb') as f:
    unique_links = pickle.load(f)

In [53]:
with open('Grupo1.BuscaPiso-NLP-Adri-n/scrap_info.pkl', 'rb') as f:
    diccionario_casas = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'Grupo1.BuscaPiso-NLP-Adri-n/scrap_info.pkl'

In [68]:
diccionario_casas = {}
i = 0
# Iteramos sobre los enlaces únicos y los usamos como claves en el diccionario
for link in unique_links:

    # Creamos un diccionario interno con una clave 'i' y un valor None
    diccionario_casas[link] = {f'{i}': {'Precio':None,
                                        'Descripcion':None,
                                        'Otras_caractericas':None,
                                        'Caracteristicas':None,
                                        'Eficiencia_energetica':None}}
    i += 1 

In [69]:
unique_links[2]

'https://www.indomio.es/anuncios/95510051/'

In [70]:
for link in unique_links:
    configurations = {
        'url': f'{link}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
    soup = BeautifulSoup(response.text, 'html.parser')    

    descripcion_elem = soup.find('div', class_='in-readAll')
    descripcion = None if descripcion_elem is None else descripcion_elem.text
    caracteristicas_titulo = soup.find_all('dl', class_='re-realEstateFeatures__list')
    print(len(caracteristicas_titulo))
    
    if len(caracteristicas_titulo) < 4:
        
        titulos = caracteristicas_titulo[0].find_all('dt', class_='re-realEstateFeatures__title')
        valores = caracteristicas_titulo[0].find_all('dd', class_='re-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[1].find('dd', class_='re-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[2].find_all('dt', class_='re-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[2].find_all('dd', class_='re-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]
        
        otr = None
    
    else:
        
        otros = caracteristicas_titulo[0].find_all('div', class_='nd-badge')
        otr = [valor.text for valor in otros]

        titulos = caracteristicas_titulo[1].find_all('dt', class_='re-realEstateFeatures__title')
        valores = caracteristicas_titulo[1].find_all('dd', class_='re-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[2].find('dd', class_='re-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[3].find_all('dt', class_='re-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[3].find_all('dd', class_='re-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]

    # Crear el diccionario para almacenar la información
    diccionario_casas[link] = {
        'Precio': precio,
        'Descripcion': descripcion,
        'Otras_caractericas': otr,
        'Caracteristicas': dict(zip(caracteristicas, values)),
        'Eficiencia_energetica': dict(zip(tit_ef, ef))
    }

4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
3
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
3
4
4
3
4
4
3
3
4
4
3
4
4
3
4
4
4
4
3
4
4
4
4
4
4
4
4
4
3
4
4
4


In [71]:
dict_iterator = iter(diccionario_casas.items())

# Get the first item
first_link, first_casa = next(dict_iterator)

# Print the details of the first house
print(f"Link: {first_link}")
print(f"Precio: {first_casa['Precio']}")
print(f"Descripcion: {first_casa['Descripcion']}")
print(f"Otras_caractericas: {first_casa['Otras_caractericas']}")
print(f"Caracteristicas: {first_casa['Caracteristicas']}")
print(f"Eficiencia_energetica: {first_casa['Eficiencia_energetica']}")

Link: https://www.indomio.es/anuncios/91743779/
Precio: € 380.000
Descripcion: Piso 188 m² en Valencia, Valencia Vivienda exterior, luminosa y muy próxima a la Avd. del Puerto y a la Avd. Baleares, rodeada de todo tipo de servicios y muy bien comunicada con transporte público. Edificio de 1960 con ascensor. Tiene 188m2 s.c y 172m2 construidos, se distribuye en hall, amplio salon comedor, 5 dormitorios, 2 baños completos con ventilación natural, amplia cocina con zona office totalmente equipada, cuarto a modo de plancha-lavadero. Trastero.Registralmente son dos viviendas unidas que mantienen las dos puertas de acceso. Por lo que se podrían segregar. Este anuncio es a titulo informativo, no vinculante ni contractual con posibles variaciones o erratas. 
Otras_caractericas: ['Armario empotrado', 'Trastero']
Caracteristicas: {'Referencia y fecha del anuncio': '3042-LO RAT PENAT - 26/05/2024', 'contrato': 'Venta', 'tipología': 'Piso', 'habitaciones': '5 dormitorios, 2 baños', 'planta': '3° p

In [72]:
next_link, next_casa = next(dict_iterator)
# Print the details of the first house
print(f"Link: {next_link}")
print(f"Precio: {next_casa['Precio']}")
print(f"Descripcion: {next_casa['Descripcion']}")
print(f"Otras_caractericas: {next_casa['Otras_caractericas']}")
print(f"Caracteristicas: {next_casa['Caracteristicas']}")
print(f"Eficiencia_energetica: {next_casa['Eficiencia_energetica']}")

Link: https://www.indomio.es/anuncios/94961259/
Precio: € 250.000
Descripcion: Espacioso piso en Valencia, ideal para parejas. La vivienda dispone de una superficie de 131 m² construidos distribuidos en salón-comedor, cocina. Se encuentra en el centro de Valencia, muy cerca de Avinguda del Professor López Piñero en el barrio de Penya-roja, cerca de Museu de les Ciències Príncipe Felipe, Gulliver park y Museu Faller de València. En sus alrededores encontramos el La Salud Hospital, la Antigua Facultad de Magisterio, supermercados, bancos, bibliotecas, farmacias, oficinas de correos, centros deportivos, varias paradas de autobús y la estación de metro Amistat-Casa de Salut. Oportunidad de inversión: inmueble procedente de ejecución hipotecaria sin posesión. Consulte con nuestro Agente Comercial la información adicional sobre el inmueble y las condiciones especiales de compraventa. El precio del inmueble ha sido fijado en atención a que el inmueble se adquiera por el comprador en estado de

In [73]:
with open('scrap_info.pkl', 'wb') as f:
    pickle.dump(diccionario_casas, f)

In [4]:
diccionario_casas

['https://www.indomio.es/anuncios/95241397/',
 'https://www.indomio.es/anuncios/91743779/',
 'https://www.indomio.es/anuncios/94961259/',
 'https://www.indomio.es/anuncios/94811643/',
 'https://www.indomio.es/anuncios/94796509/',
 'https://www.indomio.es/anuncios/95280509/',
 'https://www.indomio.es/anuncios/94416771/',
 'https://www.indomio.es/anuncios/95091529/',
 'https://www.indomio.es/anuncios/94784881/',
 'https://www.indomio.es/anuncios/95161429/',
 'https://www.indomio.es/anuncios/95226713/',
 'https://www.indomio.es/anuncios/94627545/',
 'https://www.indomio.es/anuncios/95279771/',
 'https://www.indomio.es/anuncios/94741035/',
 'https://www.indomio.es/anuncios/95311341/',
 'https://www.indomio.es/anuncios/95278941/',
 'https://www.indomio.es/anuncios/95246459/',
 'https://www.indomio.es/anuncios/95145697/',
 'https://www.indomio.es/anuncios/95229931/',
 'https://www.indomio.es/anuncios/95112247/',
 'https://www.indomio.es/anuncios/94544779/',
 'https://www.indomio.es/anuncios/

In [5]:
# Cargar el diccionario desde el archivo
with open('scrap_info.pkl', 'rb') as f:
    diccionario_casas = pickle.load(f)

EOFError: Ran out of input

In [74]:
def get_url(i,j):
    return f'https://www.indomio.es/alquiler-casas/valencia-capital/{barriadas[j]}/?pag={i}'
i = 0
j = 0
configurations = {
    'url': get_url(i,j),
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

response = requests.get(configurations['url'], headers={'User-Agent': configurations['user-agent']})

# Parsing the page
soup = BeautifulSoup(response.text, 'html.parser')
anuncios = soup.find_all('div', class_=['ad-preview'])
links = []
titulos_casas = []
anuncios = 0

for j in range(len(barriadas)):
    i = 0
    ratio = 0
    print(f'Barriada: {barriadas[j]}')
    configurations['url'] = get_url(i, j)
    
    while i <= ratio:
        print("Página actual:", configurations['url'])  
        response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})
        
        if response.status_code == 200:
            # Parsear el contenido HTML
            soup = BeautifulSoup(response.text, 'html.parser')
            anuncios_header = soup.find('div', class_='in-realEstateListHeader__title').text
            matches = re.findall(r'\d+', anuncios_header)
            # Check if any matches were found
            if matches:
                # Extract the first match and convert it to an integer
                anuncios_number = int(matches[0])
                print("Number extracted:", anuncios_number)
            else:
                anuncios_number = 0
            ratio = anuncios_number / 25
            print(f'Ratio: {ratio}')
                                
            anuncios = soup.find_all('div', class_='nd-mediaObject__content')
            for anuncio in anuncios:
                enlace = anuncio.find('a').get('href')
                time.sleep(random.randint(1, 5) / 10)
                links.append(enlace)
                titulo = anuncio.find('a').get('title')
                time.sleep(random.randint(1, 5) / 10)
                titulos_casas.append(titulo)
                
            i += 1
            configurations['url'] = get_url(i, j)

        else:
            # Si la solicitud no fue exitosa, imprimir un mensaje de error y salir del bucle
            print("Error al obtener la página:", response.status_code)
            break

Barriada: cami-fondo
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/cami-fondo/?pag=0
Ratio: 0.0
Barriada: albors
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/albors/?pag=0
Number extracted: 4
Ratio: 0.16
Barriada: la-creu-del-grau
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/la-creu-del-grau/?pag=0
Number extracted: 5
Ratio: 0.2
Barriada: penya-roja
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/penya-roja/?pag=0
Number extracted: 19
Ratio: 0.76
Barriada: aiora
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/aiora/?pag=0
Number extracted: 19
Ratio: 0.76
Barriada: ciutat-universitaria
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/ciutat-universitaria/?pag=0
Number extracted: 2
Ratio: 0.08
Barriada: mestalla
Página actual: https://www.indomio.es/alquiler-casas/valencia-capital/mestalla/?pag=0
Number extracted: 12
Ratio: 0.48
Barriada: exposicio
Pá

In [75]:
# eliminar los enlaces que sean 'https://www.indomio.es/agencias-inmobiliarias/....' ya que no son casas
links = [link for link in links if 'agencias-inmobiliarias' not in link]


In [76]:
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]
unique_links = f7(links)
unique_titles = f7(titulos_casas)

In [77]:
diccionario_casas_alq = {}
i = 0
# Iteramos sobre los enlaces únicos y los usamos como claves en el diccionario
for link in unique_links:

    # Creamos un diccionario interno con una clave 'i' y un valor None
    diccionario_casas_alq[link] = {f'{i}': {'Precio':None,
                                        'Descripcion':None,
                                        'Otras_caractericas':None,
                                        'Caracteristicas':None,
                                        'Eficiencia_energetica':None}}
    i += 1 

In [81]:
for link in unique_links:
    configurations = {
        'url': f'{link}',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}
    response = requests.get(url=configurations['url'], headers={'User-Agent': configurations['user-agent']})    
    soup = BeautifulSoup(response.text, 'html.parser')    

    descripcion_elem = soup.find('div', class_='in-readAll')
    descripcion = None if descripcion_elem is None else descripcion_elem.text
    caracteristicas_titulo = soup.find_all('dl', class_='re-realEstateFeatures__list')
    print(len(caracteristicas_titulo))
    
    if len(caracteristicas_titulo) < 4:
        
        titulos = caracteristicas_titulo[0].find_all('dt', class_='re-realEstateFeatures__title')
        valores = caracteristicas_titulo[0].find_all('dd', class_='re-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[1].find('dd', class_='re-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[2].find_all('dt', class_='re-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[2].find_all('dd', class_='re-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]
        
        otr = None
    
    else:
        
        otros = caracteristicas_titulo[0].find_all('div', class_='nd-badge')
        otr = [valor.text for valor in otros]

        titulos = caracteristicas_titulo[1].find_all('dt', class_='re-realEstateFeatures__title')
        valores = caracteristicas_titulo[1].find_all('dd', class_='re-realEstateFeatures__value')

        caracteristicas = [titulo.text for titulo in titulos]
        values = [valor.text for valor in valores]

        precio = caracteristicas_titulo[2].find('dd', class_='re-realEstateFeatures__value').text
            
        titol_ef = caracteristicas_titulo[3].find_all('dt', class_='re-realEstateFeatures__title')
        ef_v = caracteristicas_titulo[3].find_all('dd', class_='re-realEstateFeatures__value')
        tit_ef = [titulo.text for titulo in titol_ef]
        ef = [valor.text for valor in ef_v]

    # Crear el diccionario para almacenar la información
    diccionario_casas_alq[link] = {
        'Precio': precio,
        'Descripcion': descripcion,
        'Otras_caractericas': otr,
        'Caracteristicas': dict(zip(caracteristicas, values)),
        'Eficiencia_energetica': dict(zip(tit_ef, ef))
    }

4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
3
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
3
4
4
4
4
4
4
4
4
4
4
4
4
4
3
4
4
4
4
3
3
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


In [82]:
dict_iterator = iter(diccionario_casas_alq.items())

# Get the first item
first_link, first_casa = next(dict_iterator)

# Print the details of the first house
print(f"Link: {first_link}")
print(f"Precio: {first_casa['Precio']}")
print(f"Descripcion: {first_casa['Descripcion']}")
print(f"Otras_caractericas: {first_casa['Otras_caractericas']}")
print(f"Caracteristicas: {first_casa['Caracteristicas']}")
print(f"Eficiencia_energetica: {first_casa['Eficiencia_energetica']}")

Link: https://www.indomio.es/anuncios/94613177/
Precio: € 2.500/mes
Descripcion: Disponible para el alquiler MENSUAL O ANUAL. El bajo de 2 dormitorios también cuenta con una cocina totalmente equipada con electrodomésticos de alta gama, como una nevera de acero inoxidable, una cocina de inducción y un horno empotrado. Los armarios de la cocina están hechos a medida y ofrecen mucho espacio de almacenamiento. El precio de alquiler INCLUYE wifi y la limpieza semanal o bisemanal dependiendo de la estania.El salón es amplio y luminoso, con grandes ventanales que permiten la entrada de luz natural. Cuenta con un sofá de diseño y una televisión de pantalla plana. La decoración es elegante y sofisticada, con detalles como espejos de lujo y lámparas de diseño.Los dormitorios son espaciosos y cuentan con camas dobles y armarios o una zona de vestidor. Cada habitación tiene su propio aire acondicionado Split, lo que permite regular la temperatura de forma individualizada.Los baños son modernos y 

In [83]:
diccionario_casas_alq

{'https://www.indomio.es/anuncios/94613177/': {'Precio': '€ 2.500/mes',
  'Descripcion': 'Disponible para el alquiler MENSUAL O ANUAL. El bajo de 2 dormitorios también cuenta con una cocina totalmente equipada con electrodomésticos de alta gama, como una nevera de acero inoxidable, una cocina de inducción y un horno empotrado. Los armarios de la cocina están hechos a medida y ofrecen mucho espacio de almacenamiento. El precio de alquiler INCLUYE wifi y la limpieza semanal o bisemanal dependiendo de la estania.El salón es amplio y luminoso, con grandes ventanales que permiten la entrada de luz natural. Cuenta con un sofá de diseño y una televisión de pantalla plana. La decoración es elegante y sofisticada, con detalles como espejos de lujo y lámparas de diseño.Los dormitorios son espaciosos y cuentan con camas dobles y armarios o una zona de vestidor. Cada habitación tiene su propio aire acondicionado Split, lo que permite regular la temperatura de forma individualizada.Los baños son mo

In [85]:
with open('scrap_info_alq.pkl', 'wb') as f:
    pickle.dump(diccionario_casas_alq, f)